# MPI4Py - Exercises

## MPI4Py - 1st Exercise - WarmUp

Write a program that first queries the communicator's size and name, as well the processor's name and each processes' rank.

Then, each process shall print its rank and the communicator's size and name. Additionally, the root process shall indicate that it is indeed the root, e.g. by printing "I am root!".

__HINTS__: Communication is not necessary here, just two `print` statements, the variable assignments, and conditional expressions. Use `MPI.Get_processor_name()` to get the processor's name and `comm.Get_name()` to get the communicator's name.

In [ ]:
%%writefile mpi4py/mpi01_warmup.py
#!/usr/bin/python3

from mpi4py import MPI


comm = None # world communicator
size = None # commuicator size
rank = None # process rank
procname = None # processor name
commname = None # communicator name

if (True):
    print(f"I am the root process out of {size} on processor {procname} in communicator {commname}.")
else:
    print(f"I am process {rank} out of {size} on processor {procname} in communicator {commname}.")

In [ ]:
!mpirun -report-bindings --bind-to core --np 4 python3 mpi4py/mpi01_warmup.py

In [ ]:
!mpirun --oversubscribe --np 4 python3 mpi4py/mpi01_warmup.py

#### Solution Proposal

In [ ]:
%%writefile mpi4py/ex01sol_warmup.py
#!/usr/bin/python3

from mpi4py import MPI


comm = MPI.COMM_WORLD # world communicator
size = comm.Get_size() # commuicator size
rank = comm.Get_rank() # process rank
procname = MPI.Get_processor_name() # processor name
commname = comm.Get_name() # communicator name

if (rank == 0):
    print(f"I am the root process out of {size} on processor {procname} in communicator {commname}.")
else:
    print(f"I am process {rank} out of {size} on processor {procname} in communicator {commname}.")

In [ ]:
!mpiexec -report-bindings --bind-to core --np 4 python3 mpi4py/mpi01_warmup.py

In [ ]:
!mpiexec --oversubscribe --np 4 python3 mpi4py/ex01sol_warmup.py

## MPI4Py - 2nd Exercise - Point-to-Point Communication - Blocking Send & Receive

Write a program that performs a so-called __ping-pong exchange__ between __two processes__.

First the __ping__:
* Process 0 sends an empty message to process 1.
* Process 1 receives the empty message from process 0.

Then the __pong__:
* Process 1 sends an empty message to process 0.
* Process 0 receives the empty message from process 1.

__HINTS__: `comm.Send` and `comm.Recv` might be useful for this.

In [ ]:
%%writefile mpi4py/mpi02_pingpong.py
#!/usr/bin/python3

from mpi4py import MPI


comm = MPI.COMM_WORLD # communicator
rank = comm.Get_rank() # process rank
buf = None # send/receive buffer

if (True):
    partner = 1 # partner process rank
    print(f"Process {rank} sending ping to {partner}.")
    pass
    pass
    print(f"Process {rank} receiving pong from {partner}.")
elif (True):
    partner = 0 # partner process rank
    pass
    print(f"Process {rank} receiving ping from {partner}.")
    print(f"Process {rank} sending pong to {partner}.")
    pass

In [ ]:
!mpirun -report-bindings --bind-to core --np 4 python3 mpi4py/mpi02_pingpong.py

In [ ]:
!mpirun --np 2 python3 mpi4py/mpi02_pingpong.py

#### Solution Proposal

In [ ]:
%%writefile mpi4py/ex02sol_pingpong.py
#!/usr/bin/python3

import numpy as np
from mpi4py import MPI


comm = MPI.COMM_WORLD # communicator
rank = comm.Get_rank() # process rank
buf = np.empty(()) # send/receive buffer

if (rank == 0):
    partner = 1 # partner process rank
    print(f"Process {rank} sending ping to {partner}.")
    comm.Send(buf, dest=partner, tag=5)
    comm.Recv(buf, source=partner, tag=7)
    print(f"Process {rank} receiving pong from {partner}.")
elif (rank == 1):
    partner = 0 # partner process rank
    comm.Recv(buf, source=partner, tag=5)
    print(f"Process {rank} receiving ping from {partner}.")
    print(f"Process {rank} sending pong to {partner}.")
    comm.Send(buf, dest=partner, tag=7)

In [ ]:
!mpiexec -report-bindings --bind-to core --np 4 python3 mpi4py/ex02sol_pingpong.py

In [ ]:
!mpiexec --oversubscribe --np 4 python3 mpi4py/ex02sol_pingpong.py

## MPI4Py - 3rd Exercise - Collective Communication - Scatter

Scatter an array on process $0$ with numbers from $0$ to $n$ such that process $i$ receives number $i$.

In [ ]:
%%writefile mpi4py/mpi03_scatter.py
#!/usr/bin/python3

import numpy as np
from mpi4py import MPI


comm = MPI.COMM_WORLD
rank = comm.Get_rank()
commsize = comm.Get_size()

rcvbuf = np.empty([1], dtype=np.intc)
sndbuf = None

if (rank == 0):
    pass
    print(f"Process {rank} scatters {sndbuf}")

pass
    
print(f"Process {rank} receives {rcvbuf}")

In [ ]:
!mpirun -report-bindings --bind-to core --np 4 python3 mpi4py/mpi03_scatter.py

In [ ]:
!mpirun --oversubscribe --np 4 python3 mpi4py/mpi03_scatter.py

#### Solution Proposal

In [ ]:
%%writefile mpi4py/mpi03sol_scatter.py
#!/usr/bin/python3

import numpy as np
from mpi4py import MPI


comm = MPI.COMM_WORLD
rank = comm.Get_rank()
commsize = comm.Get_size()

rcvbuf = np.empty([1], dtype=np.intc)
sndbuf = None

if (rank == 0):
    sndbuf = np.arange(start=0, stop=commsize, step=1, dtype=np.intc)
    print(f"Process {rank} scatters {sndbuf}")

comm.Scatter(sndbuf, rcvbuf, root=0)
    
print(f"Process {rank} receives {rcvbuf}")

In [ ]:
!mpiexec -report-bindings --bind-to core --np 4 python3 mpi4py/mpi03sol_scatter.py

In [ ]:
!mpiexec --oversubscribe --np 4 python3 mpi4py/mpi03sol_scatter.py

## MPI4Py - 4th Exercise - Collective Communication - Allreduce

Each process holds two variables: `xi` contains its rank and `yi` contains its rank multiplied by two. Perform an all-reduction such that each process receives the sum of the products of these two variables over all processes, i.e. $s = x_0y_0 + x_1y_1 + ... x_ny_n$.

In [ ]:
%%writefile mpi4py/mpi04_allreduce.py
#!/usr/bin/python3

import numpy as np
from mpi4py import MPI


comm = MPI.COMM_WORLD
rank = comm.Get_rank()
commsize = comm.Get_size()

xi = rank
yi = 2*rank
rcvbuf = None
sndbuf = None

pass
    
print(f"Process {rank} sends {xi*yi} xi={rank}, yi={2*rank}")
print(f"Process {rank} receives {rcvbuf}")

In [ ]:
!mpirun -report-bindings --bind-to core --np 4 python3 mpi4py/mpi04_allreduce.py

In [ ]:
!mpirun --oversubscribe --np 4 python3 mpi4py/mpi04_allreduce.py

#### Solution Proposal

In [ ]:
%%writefile mpi4py/mpi04sol_allreduce.py
#!/usr/bin/python3

import numpy as np
from mpi4py import MPI


comm = MPI.COMM_WORLD
rank = comm.Get_rank()
commsize = comm.Get_size()

xi = rank
yi = 2*rank
rcvbuf = np.empty([1], dtype=np.intc)
sndbuf = np.array([xi*yi], dtype=np.intc)

comm.Allreduce(sndbuf, rcvbuf, op=MPI.SUM)
    
print(f"Process {rank} sends {xi*yi} xi={rank}, yi={2*rank}")
print(f"Process {rank} receives {rcvbuf}")

In [ ]:
!mpiexec -report-bindings --bind-to core --np 4 python3 mpi4py/mpi04sol_allreduce.py

In [ ]:
!mpiexec --oversubscribe --np 4 python3 mpi4py/mpi04sol_allreduce.py